In [467]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import math
from sklearn.linear_model import LinearRegression
from sklearn.linear_model import Ridge
from sklearn.linear_model import Lasso
CONSULTAS = r"C:\Users\moshe\OneDrive\Documentos\Machine Learning"

In [468]:
df = pd.read_csv(CONSULTAS+'\\medmal_pos.csv')
df.head(2)

,GRCODE,GRNAME,AccidentYear,DevelopmentYear,DevelopmentLag,IncurLoss_F2,CumPaidLoss_F2,BulkLoss_F2,EarnedPremDIR_F2,EarnedPremCeded_F2,EarnedPremNet_F2,Single,PostedReserve97_F2
0,669,Scpie Indemnity Co,1988,1988,1,121905,2716,97966,129104,-6214,135318,0,344558
1,669,Scpie Indemnity Co,1988,1989,2,112211,24576,64117,129104,-6214,135318,0,344558


**DESCRIPCIÓN DE LA VARIABLE**
* GRCODE NAIC: código de empresa (incluidos grupos de aseguradores y aseguradores individuales)
* GRNAME NAIC: nombre de la empresa (incluidos grupos de aseguradores y aseguradores individuales)
* AccidentYear Año del accidente(1988 a 1997)
* DevelopmentYear: Año de desarrollo (1988 a 1997)
* DevelopmentLag: Año de desarrollo (AY-1987 + DY-1987 - 1)
* IncurLoss_F2: Pérdidas incurridas y gastos asignados informados al final del año
* CumPaidLoss_F2: Pérdidas pagadas acumuladas y gastos asignados al final del año
* BulkLoss_F2: Reservas de Bulk e IBNR sobre pérdidas netas y gastos de defensa y contención de costos reportados al final del año
* EarnedPremDIR_F2: Primas ganadas en el año incurrido: directas y asumidas
* EarnedPremCeded_F2: Primas ganadas en el año incurrido - cedidas
* EarnedPremNet_F2: Primas ganadas en el año en que se produce - netas
* Single:  1 indica una sola entidad, 0 indica una aseguradora grupal
* PostedReserve97_F2: Reservas contabilizadas en el año 1997 tomadas del Anexo de Suscripción e Inversiones - Parte 2A, incluidas las pérdidas netas no pagadas y los gastos de ajuste de pérdidas no pagados

In [469]:
# 1. Descripción de las variables
variables_descripcion = df.describe(include='all')
variables_info = df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3400 entries, 0 to 3399
Data columns (total 13 columns):
 #   Column              Non-Null Count  Dtype 
---  ------              --------------  ----- 
 0   GRCODE              3400 non-null   int64 
 1   GRNAME              3400 non-null   object
 2   AccidentYear        3400 non-null   int64 
 3   DevelopmentYear     3400 non-null   int64 
 4   DevelopmentLag      3400 non-null   int64 
 5   IncurLoss_F2        3400 non-null   int64 
 6   CumPaidLoss_F2      3400 non-null   int64 
 7   BulkLoss_F2         3400 non-null   int64 
 8   EarnedPremDIR_F2    3400 non-null   int64 
 9   EarnedPremCeded_F2  3400 non-null   int64 
 10  EarnedPremNet_F2    3400 non-null   int64 
 11  Single              3400 non-null   int64 
 12  PostedReserve97_F2  3400 non-null   int64 
dtypes: int64(12), object(1)
memory usage: 345.4+ KB


In [470]:
Empresa1 = df['GRNAME'].unique()[1]
Empresa1 = df[df['GRNAME'] == Empresa1]
Empresa1

,GRCODE,GRNAME,AccidentYear,DevelopmentYear,DevelopmentLag,IncurLoss_F2,CumPaidLoss_F2,BulkLoss_F2,EarnedPremDIR_F2,EarnedPremCeded_F2,EarnedPremNet_F2,Single,PostedReserve97_F2
100,683,Promutual Grp,1988,1988,1,43393,292,15624,53178,0,53178,0,125868
101,683,Promutual Grp,1988,1989,2,39560,4697,0,53178,0,53178,0,125868
102,683,Promutual Grp,1988,1990,3,50398,13938,8431,53178,0,53178,0,125868
103,683,Promutual Grp,1988,1991,4,46630,21175,5611,53178,0,53178,0,125868
104,683,Promutual Grp,1988,1992,5,44007,24834,3813,53178,0,53178,0,125868
...,...,...,...,...,...,...,...,...,...,...,...,...,...
195,683,Promutual Grp,1997,2002,6,28330,23259,-1813,24114,1395,22719,0,125868
196,683,Promutual Grp,1997,2003,7,30665,26116,-581,24114,1395,22719,0,125868
197,683,Promutual Grp,1997,2004,8,29690,28810,-155,24114,1395,22719,0,125868
198,683,Promutual Grp,1997,2005,9,29928,29193,60,24114,1395,22719,0,125868


In [471]:
Empresa1.pivot(index='AccidentYear', columns='DevelopmentLag', values='IncurLoss_F2')

DevelopmentLag,1,2,3,4,5,6,7,8,9,10
AccidentYear,,,,,,,,,,
1988,43393,39560,50398,46630,44007,30000,27171,27395,27034,26804
1989,23460,54478,50633,46938,34432,30406,30408,29794,28014,27703
1990,44843,40405,38517,29661,23559,23269,22813,22594,21804,20984
1991,40715,44679,35510,33872,32713,30871,28734,30627,30151,30413
1992,55743,44309,33884,29723,30954,27665,27067,26018,25603,25666
1993,49446,30760,27841,37179,35687,35936,37005,38270,38382,37760
1994,33465,32438,38458,37845,35062,36552,36388,36294,36371,36409
1995,32837,32449,34650,30941,28954,29424,28768,28557,29521,29500
1996,34711,31935,27963,25959,29673,31900,33123,31773,33051,32644


In [472]:
import warnings
warnings.filterwarnings("ignore")
# Crea un pivote de datos para el triángulo de reservas
Triangulo_acumulado = Empresa1.pivot(index='AccidentYear', columns='DevelopmentLag', values='IncurLoss_F2')
for i in range(len(Triangulo_acumulado)):
    for j in range(len(Triangulo_acumulado)):
        if j < i:
            Triangulo_acumulado[i+1].iloc[len(Triangulo_acumulado)-1-j] = 0
Triangulo_acumulado = Triangulo_acumulado.replace(0,np.nan)
links_ratios = Triangulo_acumulado.applymap(lambda x: 0 if x != 0 else x)
links_ratios
for i in range(len(links_ratios)+1):
    for j in range(len(links_ratios)):
        if i > 1:
            links_ratios[i].iloc[j] = Triangulo_acumulado[i].iloc[j]/Triangulo_acumulado[i-1].iloc[j]
CSA = {}
for i in range(len(Triangulo_acumulado)-1):
    CSA[str(i+1)] = Triangulo_acumulado[i+1][0:len(Triangulo_acumulado[i+2].dropna())].sum()/Triangulo_acumulado[i+2].dropna().sum()
Triangulo_estimado = Triangulo_acumulado.copy()
for i in range(len(Triangulo_estimado)):
    for j in range(len(Triangulo_estimado)):
            if pd.isna(Triangulo_estimado[i+1].iloc[j]):
                Triangulo_estimado[i+1].iloc[j] = Triangulo_estimado[i].iloc[j]*CSA[str(i)]
Reserva_total = Reserva_total = (Triangulo_estimado[10] - list(np.diag(np.fliplr(Triangulo_estimado)))).sum()
consolidado = {'Datos_reales' : Empresa1.pivot(index='AccidentYear', columns='DevelopmentLag', values='IncurLoss_F2'),
                'Triangulo_acumulado': Triangulo_acumulado,
                'Links_ratios': links_ratios,
                'CSA': CSA,
                'Triangulo_estimado':Triangulo_estimado,
                'Reserva_total': Reserva_total}

In [473]:
for i in range(len(Triangulo_acumulado)):
    for j in range(len(Triangulo_acumulado)):
        if j < i:
            Triangulo_acumulado[i+1].iloc[len(Triangulo_acumulado)-1-j] = 0
Triangulo_acumulado = Triangulo_acumulado.replace(0,np.nan)
Triangulo_acumulado

DevelopmentLag,1,2,3,4,5,6,7,8,9,10
AccidentYear,,,,,,,,,,
1988,43393,39560.0,50398.0,46630.0,44007.0,30000.0,27171.0,27395.0,27034.0,26804.0
1989,23460,54478.0,50633.0,46938.0,34432.0,30406.0,30408.0,29794.0,28014.0,NaN
1990,44843,40405.0,38517.0,29661.0,23559.0,23269.0,22813.0,22594.0,NaN,NaN
1991,40715,44679.0,35510.0,33872.0,32713.0,30871.0,28734.0,NaN,NaN,NaN
1992,55743,44309.0,33884.0,29723.0,30954.0,27665.0,NaN,NaN,NaN,NaN
1993,49446,30760.0,27841.0,37179.0,35687.0,NaN,NaN,NaN,NaN,NaN
1994,33465,32438.0,38458.0,37845.0,NaN,NaN,NaN,NaN,NaN,NaN
1995,32837,32449.0,34650.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1996,34711,31935.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [474]:
empresas = df['GRNAME'].unique()
triangulos_empresas = {}
for empresa in empresas:
    Empresa1 = df[df['GRNAME']==empresa]
    Triangulo_acumulado = Empresa1.pivot(index='AccidentYear', columns='DevelopmentLag', values='IncurLoss_F2')
    if (np.count_nonzero(Triangulo_acumulado == 0)/Triangulo_acumulado.size*100) > 5:
        continue 
    for i in range(len(Triangulo_acumulado)):
        for j in range(len(Triangulo_acumulado)):
            if j < i:
                Triangulo_acumulado[i+1].iloc[len(Triangulo_acumulado)-1-j] = 0
    Triangulo_acumulado = Triangulo_acumulado.replace(0,np.nan)
    links_ratios = Triangulo_acumulado.applymap(lambda x: 0 if x != 0 else x)
    links_ratios
    for i in range(len(links_ratios)+1):
        for j in range(len(links_ratios)):
            if i > 1:
                links_ratios[i].iloc[j] = Triangulo_acumulado[i].iloc[j]/Triangulo_acumulado[i-1].iloc[j]
    CSA = {}
    for i in range(len(Triangulo_acumulado)-1):
        CSA[str(i+1)] = Triangulo_acumulado[i+1][0:len(Triangulo_acumulado[i+2].dropna())].sum()/Triangulo_acumulado[i+2].dropna().sum()
    Triangulo_estimado = Triangulo_acumulado.copy()
    for i in range(len(Triangulo_estimado)):
        for j in range(len(Triangulo_estimado)):
                if pd.isna(Triangulo_estimado[i+1].iloc[j]):
                    Triangulo_estimado[i+1].iloc[j] = Triangulo_estimado[i].iloc[j]*CSA[str(i)]
    Reserva_total = Reserva_total = (Triangulo_estimado[10] - list(np.diag(np.fliplr(Triangulo_estimado)))).sum()
    consolidado = {'Datos_reales' : Empresa1.pivot(index='AccidentYear', columns='DevelopmentLag', values='IncurLoss_F2'),
                    'Triangulo_acumulado': Triangulo_acumulado,
                    'Links_ratios': links_ratios,
                    'CSA': CSA,
                    'Triangulo_estimado':Triangulo_estimado,
                    'Reserva_total': Reserva_total}
    triangulos_empresas[empresa] = consolidado

In [475]:
def triangulos(data):
    empresas = df['GRNAME'].unique()
    triangulos_empresas = {}
    for empresa in empresas:
        Empresa1 = df[df['GRNAME']==empresa]
        Triangulo_acumulado = Empresa1.pivot(index='AccidentYear', columns='DevelopmentLag', values='IncurLoss_F2')
        if (np.count_nonzero(Triangulo_acumulado == 0)/Triangulo_acumulado.size*100) > 1:
            continue 
        for i in range(len(Triangulo_acumulado)):
            for j in range(len(Triangulo_acumulado)):
                if j < i:
                    Triangulo_acumulado[i+1].iloc[len(Triangulo_acumulado)-1-j] = 0
        Triangulo_acumulado = Triangulo_acumulado.replace(0,np.nan)
        links_ratios = Triangulo_acumulado.applymap(lambda x: 0 if x != 0 else x)
        links_ratios
        for i in range(len(links_ratios)+1):
            for j in range(len(links_ratios)):
                if i > 1:
                    links_ratios[i].iloc[j] = Triangulo_acumulado[i].iloc[j]/Triangulo_acumulado[i-1].iloc[j]
        CSA = {}
        for i in range(len(Triangulo_acumulado)-1):
            CSA[str(i+1)] = Triangulo_acumulado[i+1][0:len(Triangulo_acumulado[i+2].dropna())].sum()/Triangulo_acumulado[i+2].dropna().sum()
        Triangulo_estimado = Triangulo_acumulado.copy()
        for i in range(len(Triangulo_estimado)):
            for j in range(len(Triangulo_estimado)):
                    if pd.isna(Triangulo_estimado[i+1].iloc[j]):
                        Triangulo_estimado[i+1].iloc[j] = Triangulo_estimado[i].iloc[j]*CSA[str(i)]
        Reserva_total = (Triangulo_estimado[10] - list(np.diag(np.fliplr(Triangulo_estimado)))).sum()
        consolidado = {'Datos_reales' : Empresa1.pivot(index='AccidentYear', columns='DevelopmentLag', values='IncurLoss_F2'),
                       'Triangulo_acumulado': Triangulo_acumulado,
                       'Links_ratios': links_ratios,
                       'CSA': CSA,
                       'Triangulo_estimado':Triangulo_estimado,
                       'Reserva_reclamos': Reserva_total}
        triangulos_empresas[empresa] = consolidado
    return triangulos_empresas

In [476]:
triangulos_empresas = triangulos(df)

In [477]:
triangulos_empresas['Scpie Indemnity Co']['Triangulo_acumulado']

DevelopmentLag,1,2,3,4,5,6,7,8,9,10
AccidentYear,,,,,,,,,,
1988,121905,112211.0,103226.0,99599.0,96006.0,90487.0,82640.0,80406.0,78920.0,78511.0
1989,122679,113165.0,110037.0,101142.0,90817.0,81919.0,77491.0,73577.0,72716.0,NaN
1990,118157,117497.0,116377.0,99895.0,89252.0,81916.0,79134.0,76333.0,NaN,NaN
1991,117981,122443.0,121056.0,113795.0,102830.0,98071.0,94870.0,NaN,NaN,NaN
1992,131059,130155.0,124195.0,113974.0,106817.0,99182.0,NaN,NaN,NaN,NaN
1993,134700,130757.0,125253.0,114717.0,111294.0,NaN,NaN,NaN,NaN,NaN
1994,136749,128192.0,121355.0,111877.0,NaN,NaN,NaN,NaN,NaN,NaN
1995,140962,132405.0,118332.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1996,134473,128980.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [478]:
triangulos_empresas['Scpie Indemnity Co']['Triangulo_acumulado'].values.flatten()

array([121905., 112211., 103226.,  99599.,  96006.,  90487.,  82640.,
        80406.,  78920.,  78511., 122679., 113165., 110037., 101142.,
        90817.,  81919.,  77491.,  73577.,  72716.,     nan, 118157.,
       117497., 116377.,  99895.,  89252.,  81916.,  79134.,  76333.,
           nan,     nan, 117981., 122443., 121056., 113795., 102830.,
        98071.,  94870.,     nan,     nan,     nan, 131059., 130155.,
       124195., 113974., 106817.,  99182.,     nan,     nan,     nan,
           nan, 134700., 130757., 125253., 114717., 111294.,     nan,
           nan,     nan,     nan,     nan, 136749., 128192., 121355.,
       111877.,     nan,     nan,     nan,     nan,     nan,     nan,
       140962., 132405., 118332.,     nan,     nan,     nan,     nan,
           nan,     nan,     nan, 134473., 128980.,     nan,     nan,
           nan,     nan,     nan,     nan,     nan,     nan, 137944.,
           nan,     nan,     nan,     nan,     nan,     nan,     nan,
           nan,     

In [479]:
keys = list(triangulos_empresas.keys())
y = []#triangulos_empresas[keys[0]]['Triangulo_acumulado'].values.flatten()[~np.isnan(triangulos_empresas[keys[0]]['Triangulo_acumulado'].values.flatten())]
for k in keys:
    y = np.concatenate((y, triangulos_empresas[k]['Triangulo_acumulado'].values.flatten()))
y.shape

(1400,)

In [480]:
y_log = [math.log(x) for x in y]

In [481]:
x = np.zeros((triangulos_empresas[list(triangulos_empresas.keys())[0]]['Triangulo_acumulado'].shape[0]*triangulos_empresas[list(triangulos_empresas.keys())[0]]['Triangulo_acumulado'].shape[1],
              sum(triangulos_empresas[list(triangulos_empresas.keys())[0]]['Triangulo_acumulado'].shape)-2+1))
x.shape

(100, 19)

In [482]:
l = triangulos_empresas[list(triangulos_empresas.keys())[0]]['Triangulo_acumulado'].shape[0]
for i in range(x.shape[0]):
    x[i, 0] = 1
    x[i,i%l] = 1
    if i >= 10:
          entero = int(i/10)
          x[i,l+entero-1] = 1
X = x
for i in range(len(list(triangulos_empresas.keys()))-1):
     X = np.vstack((X, x))

In [483]:
x.shape, X.shape, len(y_log)

((100, 19), (1400, 19), 1400)

In [484]:
np.array(y_log).reshape(1, 1400).T.shape

(1400, 1)

In [485]:
XY_log = np.concatenate([X, np.array(y_log).reshape(1, 1400).T], axis=1)
XY_log.shape


(1400, 20)

In [486]:
df_XY_log = pd.DataFrame(XY_log)
df_XY_log = df_XY_log.dropna()
df_XY_log.shape

(770, 20)

In [487]:
x_omitnan = df_XY_log.iloc[:, :-1]
y_log_omitnan = df_XY_log.iloc[:,-1]
x_omitnan.shape,y_log_omitnan.shape

((770, 19), (770,))

In [488]:
b_est = np.dot(np.dot(np.linalg.inv(np.dot(x_omitnan.T,x_omitnan)),x_omitnan.T),y_log_omitnan)
exp_b_est = np.exp(b_est)
b_est.shape

(19,)

In [489]:
est = np.dot(x_omitnan,b_est)
exp_est = np.dot(x_omitnan,exp_b_est)

In [490]:
MAPE = 100 * np.mean(np.abs((y_log_omitnan - est) / y_log_omitnan))
MAPE


11.836502725409046

In [491]:
IBNR = sum(exp_est)
Total

NameError: name 'Total' is not defined

In [492]:
modelo = LinearRegression()
modelo.fit(X = x_omitnan, y = y_log_omitnan)
modelo.coef_.flatten(),modelo.intercept_

(array([ 0.        , -0.05366319, -0.12604845, -0.22685027, -0.28797468,
        -0.35457203, -0.39991772, -0.49217904, -0.49546519, -0.51181813,
         0.29862739,  0.39832096,  0.53248843,  0.60872173,  0.63867745,
         0.75294324,  0.81777038,  0.84446757,  0.69437918]),
 9.110288379706647)

In [493]:
b_est

array([ 9.11028838, -0.05366319, -0.12604845, -0.22685027, -0.28797468,
       -0.35457203, -0.39991772, -0.49217904, -0.49546519, -0.51181813,
        0.29862739,  0.39832096,  0.53248843,  0.60872173,  0.63867745,
        0.75294324,  0.81777038,  0.84446757,  0.69437918])

In [494]:
modelo = Ridge()
modelo.fit(X = x_omitnan, y = y_log_omitnan)
modelo.coef_,modelo.intercept_

(array([ 0.        , -0.03596196, -0.11007782, -0.21176402, -0.27376355,
        -0.34077635, -0.38673337, -0.47837044, -0.48268241, -0.50002393,
         0.25969557,  0.35823094,  0.4907095 ,  0.56527631,  0.59368066,
         0.70425592,  0.76442407,  0.78370567,  0.62576023]),
 9.13421017128364)

In [495]:
b_lineal = modelo.coef_
b_lineal[0] = modelo.intercept_
b_lineal

array([ 9.13421017, -0.03596196, -0.11007782, -0.21176402, -0.27376355,
       -0.34077635, -0.38673337, -0.47837044, -0.48268241, -0.50002393,
        0.25969557,  0.35823094,  0.4907095 ,  0.56527631,  0.59368066,
        0.70425592,  0.76442407,  0.78370567,  0.62576023])

In [496]:
triangulos_empresas['Clinic Mut Ins Co RRG']['Datos_reales']

DevelopmentLag,1,2,3,4,5,6,7,8,9,10
AccidentYear,,,,,,,,,,
1988,675,579,437,439,962,1048,1051,1049,1049,1049
1989,1370,1397,1475,1123,1144,1292,1176,1176,1176,1176
1990,1570,1437,1288,524,552,645,645,645,645,645
1991,2154,1864,2067,2519,2533,2533,2533,2533,2533,2533
1992,2721,2344,2381,1588,1685,1432,1432,1432,1432,1432
1993,3524,2649,2504,2552,2296,2050,2042,2042,2041,2041
1994,3455,4069,3906,3479,2619,2751,2776,2755,2610,2610
1995,2965,3411,3282,2564,2259,2328,1969,1969,1969,1969
1996,2855,3076,1966,1770,1713,1772,1770,1770,1770,1774


In [447]:
def regresion(triangulos_empresas):
    keys = list(triangulos_empresas.keys())
    y = []
    for k in keys:
        y = np.concatenate((y, triangulos_empresas[k]['Triangulo_acumulado'].values.flatten()))
    y_log = [math.log(x) for x in y]
    x = np.zeros((triangulos_empresas[list(triangulos_empresas.keys())[0]]['Triangulo_acumulado'].shape[0]*triangulos_empresas[list(triangulos_empresas.keys())[0]]['Triangulo_acumulado'].shape[1],
                  sum(triangulos_empresas[list(triangulos_empresas.keys())[0]]['Triangulo_acumulado'].shape)-2+1))
    l = triangulos_empresas[list(triangulos_empresas.keys())[0]]['Triangulo_acumulado'].shape[0]
    for i in range(x.shape[0]):
        x[i, 0] = 1
        x[i,i%l] = 1
        if i >= 10:
            entero = int(i/10)
            x[i,l+entero-1] = 1
    X = x
    for i in range(len(list(triangulos_empresas.keys()))-1):
        X = np.vstack((X, x))
    XY_log = np.concatenate([X, np.array(y_log).reshape(1, 1400).T], axis=1)
    df_XY_log = pd.DataFrame(XY_log)
    df_XY_log = df_XY_log.dropna()
    x_omitnan = df_XY_log.iloc[:, :-1]
    y_log_omitnan = df_XY_log.iloc[:,-1]

    ##regresion lineal
    #b_est = np.dot(np.dot(np.linalg.inv(np.dot(x_omitnan.T,x_omitnan)),x_omitnan.T),y_log_omitnan)
    #exp_b_est = np.exp(b_est)
    #exp_est = np.dot(x_omitnan,exp_b_est)
    #est = np.dot(x_omitnan,b_est)
    modelo_lineal = LinearRegression() 
    modelo_lineal.fit(X = x_omitnan, y = y_log_omitnan)
    b_est_lineal = modelo_lineal.coef_
    b_est_lineal[0] = modelo_lineal.intercept_
    est_lineal = np.dot(x_omitnan,b_est_lineal)
    #MAPE_lineal = 100 * np.mean(np.abs((y_log_omitnan - est_lineal) / y_log_omitnan))

    ##regresion Ridge
    modelo_ridge = Ridge()
    modelo_ridge.fit(X = x_omitnan, y = y_log_omitnan)
    b_est_ridge = modelo_ridge.coef_
    b_est_ridge[0] = modelo_ridge.intercept_
    est_ridge = np.dot(x_omitnan,b_est_ridge)
    #MAPE_ridge = 100 * np.mean(np.abs((y_log_omitnan - est_ridge) / y_log_omitnan))

    ##regresion Lasso
    modelo_lasso = Lasso()
    modelo_lasso.fit(X = x_omitnan, y = y_log_omitnan)
    b_est_lasso = modelo_lasso.coef_
    b_est_lasso[0] = modelo_lasso.intercept_
    est_lasso = np.dot(x_omitnan,b_est_lasso)
    #MAPE_lasso = 100 * np.mean(np.abs((y_log_omitnan - est_lasso) / y_log_omitnan))
    #     
    #MAPE = {'b_est_lineal':MAPE_lineal,'b_est_ridge':MAPE_ridge,'b_est_lasso':MAPE_lasso}
    #modelo_optimo = locals()[min(MAPE, key=MAPE.get)]
    modelo_optimo = [b_est_lineal,b_est_ridge,b_est_lasso,x_omitnan.index]
    return modelo_optimo        

In [506]:
x = np.zeros((triangulos_empresas[list(triangulos_empresas.keys())[0]]['Triangulo_acumulado'].shape[0]*triangulos_empresas[list(triangulos_empresas.keys())[0]]['Triangulo_acumulado'].shape[1],
                sum(triangulos_empresas[list(triangulos_empresas.keys())[0]]['Triangulo_acumulado'].shape)-2+1))
l = triangulos_empresas[list(triangulos_empresas.keys())[0]]['Triangulo_acumulado'].shape[0]
for i in range(x.shape[0]):
    x[i, 0] = 1
    x[i,i%l] = 1
    if i >= 10:
        entero = int(i/10)
        x[i,l+entero-1] = 1
X = x
for i in range(len(list(triangulos_empresas.keys()))-1):
    X = np.vstack((X, x))
def regresion(triangulos_empresas,X):
    keys = list(triangulos_empresas.keys())
    y = []
    for k in keys:
        y = np.concatenate((y, triangulos_empresas[k]['Triangulo_acumulado'].values.flatten()))
    y_log = [math.log(x) for x in y]
    XY_log = np.concatenate([X, np.array(y_log).reshape(1, X.shape[0]).T], axis=1)
    df_XY_log = pd.DataFrame(XY_log)
    df_XY_log = df_XY_log.dropna()
    x_omitnan = df_XY_log.iloc[:, :-1]
    y_log_omitnan = df_XY_log.iloc[:,-1]

    ##regresion lineal
    modelo_lineal = LinearRegression() 
    modelo_lineal.fit(X = x_omitnan, y = y_log_omitnan)
    b_est_lineal = modelo_lineal.coef_
    b_est_lineal[0] = modelo_lineal.intercept_
    est_lineal = np.dot(x_omitnan,b_est_lineal)

    ##regresion Ridge
    modelo_ridge = Ridge()
    modelo_ridge.fit(X = x_omitnan, y = y_log_omitnan)
    b_est_ridge = modelo_ridge.coef_
    b_est_ridge[0] = modelo_ridge.intercept_
    est_ridge = np.dot(x_omitnan,b_est_ridge)

    ##regresion Lasso
    modelo_lasso = Lasso()
    modelo_lasso.fit(X = x_omitnan, y = y_log_omitnan)
    b_est_lasso = modelo_lasso.coef_
    b_est_lasso[0] = modelo_lasso.intercept_
    est_lasso = np.dot(x_omitnan,b_est_lasso)

    modelo_optimo = [b_est_lineal,b_est_ridge,b_est_lasso,x_omitnan.index]
    return modelo_optimo        

In [441]:
y = triangulos_empresas['Scpie Indemnity Co']['Datos_reales'].values.flatten()
y_log = [math.log(x) for x in y]
XY_log = np.concatenate([x, np.array(y_log).reshape(1, 100).T], axis=1)
XY_log = pd.DataFrame(XY_log).drop(regresion(triangulos_empresas,X)[3][regresion(triangulos_empresas,X)[3]<100])
x_test = XY_log.iloc[:, :-1]
y_test = XY_log.iloc[:, -1]
est_lineal = np.dot(x_test,regresion(triangulos_empresas,X)[0])
MAPE_lineal = 100 * np.mean(np.abs((y_test - est_lineal) / y_test))
MAPE_lineal


17.86256544835233

In [538]:
def validacion(triangulos_empresas):
    triangulos = list(triangulos_empresas.keys())
    x = np.zeros((triangulos_empresas[list(triangulos_empresas.keys())[0]]['Triangulo_acumulado'].shape[0]*triangulos_empresas[list(triangulos_empresas.keys())[0]]['Triangulo_acumulado'].shape[1],
                    sum(triangulos_empresas[list(triangulos_empresas.keys())[0]]['Triangulo_acumulado'].shape)-2+1))
    l = triangulos_empresas[list(triangulos_empresas.keys())[0]]['Triangulo_acumulado'].shape[0]
    for i in range(x.shape[0]):
        x[i, 0] = 1
        x[i,i%l] = 1
        if i >= 10:
            entero = int(i/10)
            x[i,l+entero-1] = 1
    X = x
    for i in range(len(list(triangulos_empresas.keys()))-2):
        X = np.vstack((X, x))
    for i in range(len(list(triangulos_empresas.keys()))-1):
        X_0 = np.vstack((X, x))
    regresion0 = regresion(triangulos_empresas,X_0)

    best_i = []
    '''  best = []
    for j in triangulos:
        triangulos_empresas_j = triangulos_empresas.copy()
        del triangulos_empresas_j[j]
        triangulos_j = list(triangulos_empresas.keys())'''
    for i in triangulos:
        print(i)
        triangulos_empresas_i = triangulos_empresas.copy()
        del triangulos_empresas_i[i]
        y = triangulos_empresas[i]['Datos_reales'].values.flatten()
        y_log = [math.log(x) for x in y]
        XY_log = np.concatenate([x, np.array(y_log).reshape(1, 100).T], axis=1)
        XY_log = pd.DataFrame(XY_log).drop(regresion0[3][regresion0[3]<100])
        x_test = XY_log.iloc[:, :-1]
        y_test = XY_log.iloc[:, -1]
        regresion_i = regresion(triangulos_empresas_i,X)
        est_lineal = np.dot(x_test,regresion_i[0])
        est_ridge = np.dot(x_test,regresion_i[1])
        est_lasso = np.dot(x_test,regresion_i[2])
        
        MAPE_lineal = 100 * np.mean(np.abs((y_test - est_lineal) / y_test))
        MAPE_ridge = 100 * np.mean(np.abs((y_test - est_ridge) / y_test)) 
        MAPE_lasso = 100 * np.mean(np.abs((y_test - est_lasso) / y_test))
        MAPE = {'est_lineal':MAPE_lineal,'est_ridge':MAPE_ridge,'est_lasso':MAPE_lasso}
        modelo_optimo = locals()[min(MAPE, key=MAPE.get)]
        best_i.append(modelo_optimo)
    
    return best_i

    

In [541]:
x = validacion(triangulos_empresas)

Scpie Indemnity Co
Promutual Grp
Markel Corp Grp
Texas Hospital Ins Exch
State Volunteer Mut Ins Co
MHA Ins Co
Preferred Professional Ins Co
Medical Mut Ins Co Of ME
Utah Medical Ins Assoc
Seguros Triples Inc
Dentists Ins Co


Physicians Recip Insurers
Louisiana Med Mut Ins Co
Clinic Mut Ins Co RRG


14

In [513]:
def cross_validation(triangulos_empresas):
    triangulos = list(triangulos_empresas.keys())
    x = np.zeros((triangulos_empresas[list(triangulos_empresas.keys())[0]]['Triangulo_acumulado'].shape[0]*triangulos_empresas[list(triangulos_empresas.keys())[0]]['Triangulo_acumulado'].shape[1],
                    sum(triangulos_empresas[list(triangulos_empresas.keys())[0]]['Triangulo_acumulado'].shape)-2+1))
    l = triangulos_empresas[list(triangulos_empresas.keys())[0]]['Triangulo_acumulado'].shape[0]
    for i in range(x.shape[0]):
        x[i, 0] = 1
        x[i,i%l] = 1
        if i >= 10:
            entero = int(i/10)
            x[i,l+entero-1] = 1
    X = x
    for i in range(len(list(triangulos_empresas.keys()))-2):
        X = np.vstack((X, x))
    for i in range(len(list(triangulos_empresas.keys()))-1):
        X_0 = np.vstack((X, x))
    regresion0 = regresion(triangulos_empresas,X_0)

    best_i = []
    best = []
    for j in triangulos:
        triangulos_empresas_j = triangulos_empresas.copy()
        del triangulos_empresas_j[j]
        triangulos_j = list(triangulos_empresas.keys())
        for i in triangulos:
            print(i)
            triangulos_empresas_i = triangulos_empresas.copy()
            del triangulos_empresas_i[i]
            y = triangulos_empresas[i]['Datos_reales'].values.flatten()
            y_log = [math.log(x) for x in y]
            XY_log = np.concatenate([x, np.array(y_log).reshape(1, 100).T], axis=1)
            XY_log = pd.DataFrame(XY_log).drop(regresion0[3][regresion0[3]<100])
            x_test = XY_log.iloc[:, :-1]
            y_test = XY_log.iloc[:, -1]
            regresion_i = regresion(triangulos_empresas_i,X)
            est_lineal = np.dot(x_test,regresion_i[0])
            est_ridge = np.dot(x_test,regresion_i[1])
            est_lasso = np.dot(x_test,regresion_i[2])
            
            MAPE_lineal = 100 * np.mean(np.abs((y_test - est_lineal) / y_test))
            MAPE_ridge = 100 * np.mean(np.abs((y_test - est_ridge) / y_test)) 
            MAPE_lasso = 100 * np.mean(np.abs((y_test - est_lasso) / y_test))
            MAPE = {'est_lineal':MAPE_lineal,'est_ridge':MAPE_ridge,'est_lasso':MAPE_lasso}
            modelo_optimo = locals()[min(MAPE, key=MAPE.get)]
            best_i.append(modelo_optimo)

            
        
    
    return best_i

In [514]:
regresion_i[1]

array([ 9.13421017, -0.03596196, -0.11007782, -0.21176402, -0.27376355,
       -0.34077635, -0.38673337, -0.47837044, -0.48268241, -0.50002393,
        0.25969557,  0.35823094,  0.4907095 ,  0.56527631,  0.59368066,
        0.70425592,  0.76442407,  0.78370567,  0.62576023])

## Estimación por regresión omitiendo NA

In [13]:
triangulos_empresas['Scpie Indemnity Co']['Triangulo_acumulado']

DevelopmentLag,1,2,3,4,5,6,7,8,9,10
AccidentYear,,,,,,,,,,
1988,121905,112211.0,103226.0,99599.0,96006.0,90487.0,82640.0,80406.0,78920.0,78511.0
1989,122679,113165.0,110037.0,101142.0,90817.0,81919.0,77491.0,73577.0,72716.0,NaN
1990,118157,117497.0,116377.0,99895.0,89252.0,81916.0,79134.0,76333.0,NaN,NaN
1991,117981,122443.0,121056.0,113795.0,102830.0,98071.0,94870.0,NaN,NaN,NaN
1992,131059,130155.0,124195.0,113974.0,106817.0,99182.0,NaN,NaN,NaN,NaN
1993,134700,130757.0,125253.0,114717.0,111294.0,NaN,NaN,NaN,NaN,NaN
1994,136749,128192.0,121355.0,111877.0,NaN,NaN,NaN,NaN,NaN,NaN
1995,140962,132405.0,118332.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1996,134473,128980.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [14]:
triangulos_empresas['Scpie Indemnity Co']['Triangulo_acumulado'].values.flatten()[~np.isnan(triangulos_empresas['Scpie Indemnity Co']['Triangulo_acumulado'].values.flatten())]

array([121905., 112211., 103226.,  99599.,  96006.,  90487.,  82640.,
        80406.,  78920.,  78511., 122679., 113165., 110037., 101142.,
        90817.,  81919.,  77491.,  73577.,  72716., 118157., 117497.,
       116377.,  99895.,  89252.,  81916.,  79134.,  76333., 117981.,
       122443., 121056., 113795., 102830.,  98071.,  94870., 131059.,
       130155., 124195., 113974., 106817.,  99182., 134700., 130757.,
       125253., 114717., 111294., 136749., 128192., 121355., 111877.,
       140962., 132405., 118332., 134473., 128980., 137944.])

In [15]:
keys = list(triangulos_empresas.keys())
y = []#triangulos_empresas[keys[0]]['Triangulo_acumulado'].values.flatten()[~np.isnan(triangulos_empresas[keys[0]]['Triangulo_acumulado'].values.flatten())]
for k in keys:
    y = np.concatenate((y, triangulos_empresas[k]['Triangulo_acumulado'].values.flatten()[~np.isnan(triangulos_empresas[k]['Triangulo_acumulado'].values.flatten())]))
y.shape

(770,)

In [16]:
triangulos_empresas[list(triangulos_empresas.keys())[0]]['Triangulo_acumulado'].values.flatten()[~np.isnan(triangulos_empresas[list(triangulos_empresas.keys())[0]]['Triangulo_acumulado'].values.flatten())]

array([121905., 112211., 103226.,  99599.,  96006.,  90487.,  82640.,
        80406.,  78920.,  78511., 122679., 113165., 110037., 101142.,
        90817.,  81919.,  77491.,  73577.,  72716., 118157., 117497.,
       116377.,  99895.,  89252.,  81916.,  79134.,  76333., 117981.,
       122443., 121056., 113795., 102830.,  98071.,  94870., 131059.,
       130155., 124195., 113974., 106817.,  99182., 134700., 130757.,
       125253., 114717., 111294., 136749., 128192., 121355., 111877.,
       140962., 132405., 118332., 134473., 128980., 137944.])

In [17]:
x = np.zeros((len(triangulos_empresas[list(triangulos_empresas.keys())[0]]['Triangulo_acumulado'].values.flatten()[~np.isnan(triangulos_empresas[list(triangulos_empresas.keys())[0]]['Triangulo_acumulado'].values.flatten())]), 
              sum(triangulos_empresas[list(triangulos_empresas.keys())[0]]['Triangulo_acumulado'].shape)-2+1))
x.shape

(55, 19)

In [182]:
x = np.zeros((55,sum(triangulos_empresas[list(triangulos_empresas.keys())[0]]['Triangulo_acumulado'].shape)-2+1))
l = 10
for i in range(x.shape[0]):
    x[i, 0] = 1
    x[i,i%l] = 1
    if i > 10:
        if (i/(l+2) - int(i/(l+2))) == 0:
            l = l-1

            

In [232]:
pd.DataFrame(x)

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18
0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
5,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
6,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
7,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
8,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
9,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [21]:
10+9+8+7+6+5+4+3+2+1

55